In [ ]:
!pip install basemap
!pip install basemap-data-hires
!pip install geopandas
!pip install openturns

In [ ]:
import numpy as np
import pandas as pd
import math
import copy
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)
from mpl_toolkits.basemap import Basemap
import geopandas as gpd
from scipy.spatial import ConvexHull
from timeit import default_timer as timer
from datetime import datetime
from datetime import timedelta
import openturns as ot
from openturns.viewer import View

In [ ]:
from aux_code.learning_preprocess import convert_from_NED_to_Robotic, get_trajectory_before_pass, check_dimension, get_transmat_g_wrt_r

In [ ]:
def get_cartesian(lat=None,lon=None):
    # https://stackoverflow.com/a/55256861
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    R = 6371 # radius of the earth (km)
    x = R * np.cos(lat) * np.cos(lon) * 1000 # (m)
    y = R * np.cos(lat) * np.sin(lon) * 1000 # (m)
    # z = R *np.sin(lat)
    return x,y

In [ ]:
def check_vector_angle(vec1, vec2):
    # type: (np.array, np.array) -> float
    """
    function to return vector angle by vec1 as a base
    https://math.stackexchange.com/questions/878785/how-to-find-an-angle-in-range0-360-between-2-vectors/879474

    Arguments: 
        - vec1, vec2 format np.array([x,y]) 1x2, i.e., dim (2,)

    Returns: 
        - angle (radian): [-math.pi, math.pi]
    """

    # dot product and determinant    
    dot_product = vec1[0] * vec2[0] + vec1[1] * vec2[1]
    determinant = vec1[0] * vec2[1] - vec1[1] * vec2[0]
    angle = math.atan2(determinant, dot_product)

    return angle

### Data read test

### parameters

In [ ]:
# Boundaries
km_to_m = 1000 # if use km put 1


# left_x = 2000 * km_to_m
# right_x = 4000 * km_to_m
# upper_y = 700 * km_to_m
# lower_y = 300 * km_to_m


left_x = 3565989.1586356782
right_x = 3780904.649607267
upper_y = 452316.00 
lower_y = 110233.24 

spd_constraint = 0.2
encounter_dist = 2 * km_to_m
sampling_time = 10 # seconds
effective_encounter_dist = 3 * 1852 # nm 
clearing_cond_dist = 3 * 1000 # 1 km 
clearing_cond_angle = 112.5 # deg

finish_finding_ships = 200
bow_crossing_hit_time = 3 # ea
SPD_THRE = 5 #knots
MONITORING_DIST = 3 * km_to_m
FINAL_OUT = 20 * 1852
ONLY_FOWARD_COME = True

* trajectory linear interpolation
* one ship --> other ships perspective

In [ ]:
def read_csv_ais(csv_name):
    """
    Args:
        csv_name: e.g., 20180101 
    
    Returns:
        gps_processed_df
    """
    #####################################################
    # data loading
    #####################################################
    # date_df = pd.read_csv("./dk_ais/dk_csv_{}/aisdk_{}.csv".format(csv_name, csv_name), encoding='unicode_escape')
    # date_df = pd.read_csv(csv_name, encoding='unicode_escape')
    date_df = pd.read_csv(csv_name)

    print("AIS data successfully loaded")
    # print(date_df.head())


    #####################################################
    # pre-processing
    #####################################################
    gps_processed_df = date_df.copy()
    gps_processed_df['pos_x'], gps_processed_df['pos_y'] = get_cartesian(date_df['Latitude'], date_df['Longitude'])
    gps_processed_df.columns = gps_processed_df.columns.str.replace('# ', '')

    # crop the region and non-valid value
    gps_processed_df = gps_processed_df.loc[(right_x >= gps_processed_df['pos_x']) 
                                            & (gps_processed_df['pos_x'] >= left_x) 
                                            & (upper_y >= gps_processed_df['pos_y'])
                                            & (gps_processed_df['pos_y'] >= lower_y)].dropna(subset=['Timestamp','MMSI', 'Heading', 'SOG', 'COG'])
    gps_processed_df['Timestamp'] =  pd.to_datetime(gps_processed_df['Timestamp'])
    gps_processed_df['Timestamp'] =  gps_processed_df['Timestamp'].dt.strftime('%d/%m/%y %H:%M:%S')
    gps_processed_df['heading_converted'] =  np.radians(gps_processed_df['Heading']).apply(convert_from_NED_to_Robotic)

    print(gps_processed_df.head())
    return gps_processed_df

### data obtain test

In [ ]:
def get_final_dataframe(gps_processed_df, csv_name, other_idx, total_desire_num):

    found = False
    df_entire_pass = pd.DataFrame()

    for ego_id, df_by_ego_ID in gps_processed_df.groupby('MMSI'):
        
        """ ego vehicle """
        # edge case where it is not using engine
        if (df_by_ego_ID['Navigational status'] != 'Under way using engine').any():
            continue
        elif (df_by_ego_ID['SOG'] <= SPD_THRE).any():
            continue

        # 1) timestamp reindex for interpolate
        df_by_ego_ID['Timestamp'] =  pd.to_datetime(df_by_ego_ID['Timestamp'], format='%d/%m/%y %H:%M:%S')
        df_by_ego_ID = df_by_ego_ID.set_index('Timestamp')

        # 2) filling the gap
        df_ego_interpol = df_by_ego_ID.resample("{}s".format(str(sampling_time))).mean(numeric_only=True) # mean or padding any other method?
        df_ego_interpol = df_ego_interpol.interpolate() 
        # TODO linear or something? https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.interpolate.html
        # method='spline'

        # print(df_ego_interpol)


        """ other vehicle """
        for obj_id, df_by_obj_ID in gps_processed_df.groupby('MMSI'):
            if other_idx >= total_desire_num:
                return other_idx

            # edge case: not ego vehicle
            if obj_id == ego_id:
                continue
            elif (df_by_obj_ID['SOG'] <= SPD_THRE).any():
                continue

            # 1) timestamp reindex for interpolate
            df_by_obj_ID['Timestamp'] =  pd.to_datetime(df_by_obj_ID['Timestamp'], format='%d/%m/%y %H:%M:%S')
            df_by_obj_ID = df_by_obj_ID.set_index('Timestamp')

            # 2) filling the gap
            df_obj_interpol = df_by_obj_ID.resample("{}s".format(str(sampling_time))).mean(numeric_only=True) # mean or padding any other method?
            df_obj_interpol = df_obj_interpol.interpolate() # TODO linear or something?

            # 3) find intersection
            # https://sparkbyexamples.com/pandas/find-intersection-between-two-series-in-pandas/?expand_article=1
            # time_intersect = set(df_ego_interpol.index) & set(df_obj_interpol.index)
            time_intersect = df_ego_interpol.index.intersection(df_obj_interpol.index)

            # 4) find closest distance meeting
            if len(time_intersect):
                ############################# 
                # crop data processing 
                #############################


                """ actual build of DataFrame """
                df_rel_obj = pd.DataFrame()
                
                df_rel_obj['global_x'] = df_obj_interpol.loc[time_intersect]['pos_x'] - df_ego_interpol.loc[time_intersect]['pos_x']
                df_rel_obj['global_y'] = df_obj_interpol.loc[time_intersect]['pos_y'] - df_ego_interpol.loc[time_intersect]['pos_y']
                df_rel_obj['global_v_x'] = df_rel_obj['global_x'].diff() / sampling_time # relative vel x
                df_rel_obj['global_v_y'] = df_rel_obj['global_y'].diff() / sampling_time # relative vel x
                comp_bearing = np.arctan2(df_rel_obj['global_y'], df_rel_obj['global_x'])
                # transformation
                # ----------------------------------------------------------------
                xy_rel_global = np.vstack([df_rel_obj['global_x'].to_numpy(), 
                                            df_rel_obj['global_y'].to_numpy()])
                xy_rel_global = np.vstack([xy_rel_global, np.zeros(len(df_rel_obj['global_x']))])
                xy_rel_global = np.vstack([xy_rel_global, np.ones(len(df_rel_obj['global_x']))])

                df_rel_obj['TF_matrix'] = df_ego_interpol.loc[time_intersect]['heading_converted'].apply(get_transmat_g_wrt_r)
                matrix_mul_array_x =[]
                matrix_mul_array_y =[]
                for mat_idx in range(len(df_rel_obj['TF_matrix'])):
                    mattrix_mul_result = np.matmul(df_rel_obj['TF_matrix'][mat_idx], xy_rel_global[:, mat_idx])
                    matrix_mul_array_x.append(mattrix_mul_result[0])
                    matrix_mul_array_y.append(mattrix_mul_result[1])
                # ----------------------------------------------------------------
                df_rel_obj['x'] = matrix_mul_array_x # relative x
                df_rel_obj['y'] = matrix_mul_array_y # relative y
                df_rel_obj['v_x'] = df_rel_obj['x'].diff() / sampling_time # relative vel x
                df_rel_obj['v_y'] = df_rel_obj['y'].diff() / sampling_time # relative vel y
                df_rel_obj['rel_dist'] = np.sqrt(df_rel_obj['global_x']**2 + df_rel_obj['global_y']**2)
                df_rel_obj['rel_bearing'] = np.arctan2(df_rel_obj['y'], df_rel_obj['x']) # atan wrt my vehicle
                df_rel_obj['rel_bearing_diff'] = df_rel_obj['rel_bearing'].diff()
                df_rel_obj[df_rel_obj['rel_bearing_diff'] > np.pi] -= np.pi * 2
                df_rel_obj[df_rel_obj['rel_bearing_diff'] < -np.pi] += np.pi * 2
                df_rel_obj['comp_bearing'] = comp_bearing
                df_rel_obj['comp_bearing_diff'] = df_rel_obj['comp_bearing'].diff()
                df_rel_obj[df_rel_obj['comp_bearing_diff'] > np.pi] -= np.pi * 2
                df_rel_obj[df_rel_obj['comp_bearing_diff'] < -np.pi] += np.pi * 2
                df_rel_obj['Heading'] = df_obj_interpol['Heading']
                df_rel_obj['heading_converted'] =  np.radians(df_rel_obj['Heading']).apply(convert_from_NED_to_Robotic)
                df_rel_obj['sin_rel_bearing'] = np.sin(df_rel_obj['rel_bearing'] )
                df_rel_obj['cos_rel_bearing'] = np.cos(df_rel_obj['rel_bearing'] )
                df_rel_obj['sin_comp_bearing'] = np.sin(df_rel_obj['comp_bearing'])
                df_rel_obj['cos_comp_bearing'] = np.cos(df_rel_obj['comp_bearing'])
                df_rel_obj['sin_heading_converted'] = np.sin(df_obj_interpol.loc[time_intersect]['heading_converted'])
                df_rel_obj['cos_heading_converted'] = np.cos(df_obj_interpol.loc[time_intersect]['heading_converted'])   
                df_rel_obj.drop(['TF_matrix'], inplace=True, axis=1)
                # print(df_rel_obj)
        

                # ------------------------------------------------------------------
                # within distance threshold
                encounter_cond = (df_rel_obj['rel_dist'] <= encounter_dist).any()
                """ encounter detail condition crop """
                # meaning encounter as below a certain distance  
                if encounter_cond: 
                    #################################################################################################################### 
                    # 0) pre-processing to find whether actually passing happened
                    ####################################################################################################################

                    crop_start_idx = df_rel_obj.index[df_rel_obj['rel_dist'] <= MONITORING_DIST][0]
                    df_rel_obj_crop = (df_rel_obj.loc[crop_start_idx:].copy()).reset_index(drop=True) # Indexing time --> loc

                    # print("cropped", df_rel_obj_crop)
                    end_idx = 0
                    enter_idx = 0 
                    clear_idx = 0
                    entered_flag = False

                    for k in range(len(df_rel_obj_crop)):
                        rel_dist_current = df_rel_obj_crop['rel_dist'].iloc[k]
                        rel_bearing_current = df_rel_obj_crop['rel_bearing'].iloc[k]

                        if not entered_flag and rel_dist_current <= encounter_dist:
                            """ entering condition """
                            entered_flag = True

                        elif entered_flag:
                            """ clearing condition """

                            # LOS is based on global, not relative wrt my ASV
                            global_LOS_vec = np.array([df_rel_obj_crop['global_x'].iloc[k], df_rel_obj_crop['global_y'].iloc[k]])

                            # TODO maybe speed * hdg?
                            global_rel_vec_wrt_other = np.array([-df_rel_obj_crop['global_v_x'].iloc[k],
                                                        -df_rel_obj_crop['global_v_y'].iloc[k]])

                            dot_product = np.dot(global_rel_vec_wrt_other, global_LOS_vec)
                            # print("rel dist {} rel bear {} dot {}".format(rel_dist_current, rel_bearing_current, dot_product))
                            
                            intersect_angle = check_vector_angle(global_rel_vec_wrt_other, global_LOS_vec) 

                            if (dot_product < 0 and \
                            rel_dist_current >= clearing_cond_dist and \
                            abs(intersect_angle) >= np.radians(clearing_cond_angle)):
                                clear_idx = k
                                break




                    #################################################################################################################### 
                    # 1) valid encounter find: approaching from the front side
                    ####################################################################################################################
                    if clear_idx != 0:
                        # 1) pos_x extraction to extract coming from front
                        df_rel_obj_crop_pass = df_rel_obj_crop.loc[:clear_idx].copy()

                        ###################################
                        ######### OPTIONS for crop out
                        df_rel_obj_crop_use = df_rel_obj_crop # 1) from enter to end
                        # df_rel_obj_crop_use = df_rel_obj_crop_pass # 2) from enter to clear
                        # df_rel_obj_crop_use = df_rel_obj # 3) all trajectory
                        df_rel_obj_crop_use = df_rel_obj_crop_use.reset_index(drop=True)
                        
                        # only monitoring inside end part crop
                        df_rel_obj_crop_use = df_rel_obj_crop_use[df_rel_obj_crop_use["rel_dist"] < FINAL_OUT]
                        df_rel_obj_crop_use = df_rel_obj_crop_use.reset_index(drop=True)
                        # print("df_rel_obj_crop_use {}".format(df_rel_obj_crop_use))
                        ###################################

                        init_pos_x = df_rel_obj_crop_pass['x'].iloc[0] # init position as enter
                        last_pos_x = df_rel_obj_crop['x'].iloc[-1] # final position till the end
                        init_pos_y = df_rel_obj_crop_pass['y'].iloc[0] # init position as enter
                        last_pos_y = df_rel_obj_crop['y'].iloc[-1] # final position till the end



                        ###################################
                        # coming from the aft
                        ###################################
                        if init_pos_x <= 0: 
                            if ONLY_FOWARD_COME:
                                continue # only from front 
                            else:
                                pass
                        
                            # 1) overtaken
                            if last_pos_x >=0: # last position 1,4 quadrant
                                df_rel_obj_crop_use['valid'] = True
                                df_rel_obj_crop_use['obj_index'] = other_idx
                            
                            # 2) crossing but passing behind
                            else: # last position also 2,3 quadrant
                                if init_pos_y <= 0: # 3rd quadrant
                                    # 2-1) check it goes above x
                                    if last_pos_y >= 0: # passing and clear condition already checked by dot and rel-bearing above
                                        df_rel_obj_crop_use['valid'] = True
                                        df_rel_obj_crop_use['obj_index'] = other_idx
                                    # 2-2) check it pass behind certain distance (last output)
                                    elif np.sqrt(np.power(last_pos_x,2) + np.power(last_pos_y,2)) >= MONITORING_DIST:
                                        df_rel_obj_crop_use['valid'] = True
                                        df_rel_obj_crop_use['obj_index'] = other_idx
                                    else:
                                        continue
                                else: # 2nd quadrant
                                    # 2-1) check it goes below x
                                    if last_pos_y <= 0: # passing and clear condition already checked by dot and rel-bearing above
                                        df_rel_obj_crop_use['valid'] = True
                                        df_rel_obj_crop_use['obj_index'] = other_idx
                                    elif np.sqrt(np.power(last_pos_x,2) + np.power(last_pos_y,2)) >= MONITORING_DIST:
                                        df_rel_obj_crop_use['valid'] = True
                                        df_rel_obj_crop_use['obj_index'] = other_idx
                                    else:
                                        continue                          

                        ###################################
                        # coming from the front
                        ###################################
                        else: # init_pose >= 0
                            # 3-1) either crossing from forward (can be bow crossing or not)
                            # 3-2) head-on
                            if last_pos_x <= 0: # ended up passing 
                                # TODO this return and TEST sanity check
                                df_rel_obj_crop_use['valid'] = True
                                df_rel_obj_crop_use['obj_index'] = other_idx
                                
                            # 4) didn't pass and cleared my beam
                            else: 
                                # it never pass my abeam --> should be skipped
                                continue
                                
                        print("MMSI {} found valid ....... ".format(other_idx))

                        #################################################################################################################### 
                        # 2) angle difference, r, and winding number
                        ####################################################################################################################
                        if bool(df_rel_obj_crop_use['valid'].iloc[0]):
                            # passing homotopy
                            winding_number = df_rel_obj_crop_use['comp_bearing_diff'].sum()
                            # winding_number = df_rel_obj_crop_use['rel_bearing_diff'].sum()
                            print("winding number {}".format(winding_number))
                            label = None
                            if winding_number > 0:
                                # print("passing on my L side")
                                df_rel_obj_crop_use['label'] = 'L' # entire cell mapping
                                label = 'L'
                            
                            elif winding_number < 0:
                                # print("passing on my R side")
                                df_rel_obj_crop_use['label'] = 'R' # entire cell mapping
                                label = 'R'

                            df_rel_obj_crop_use.reset_index(drop=True)
                            df_rel_obj_crop_use.to_pickle("./extracted_data/aisdk_{}.pkl".format(other_idx))
                            print("{} th object saved label {}.......{} aisdk_{}.pkl".format(other_idx,label, csv_name, other_idx))

                            # df_rel_obj_crop_use.to_pickle("./dk_csv_{}/aisdk_{}_{}.pkl".format(csv_name, csv_name, other_idx))
                            # print("{} th object saved label {}....... aisdk_{}_{}.pkl".format(other_idx,label, csv_name, other_idx))
                            print("-------------------------------------------------------------")
                        
                            other_idx += 1
    return other_idx
    # print("final {}".format(other_idx))

In [ ]:
import os
def find_all_csvs(path):
    ##### iteration of all bag files
    csv_files = []
    for r, d, f in os.walk(path):
        for file in f:
            if '.csv' in file:
                csv_files.append(os.path.join(r, file))

    return csv_files

### Main function to find

In [ ]:
def wrapping_function_obtain_data(total_desire_num):
    global other_idx
    other_idx = 110
    # label_l_total = 0
    # label_r_total = 0
    # desired_half = total_desire_num / 2
    csv_name_arr = sorted(find_all_csvs('./dk_ais/'))[20:]
    # TODO file name loop
    print("starting generation real AIS .............")
    for csv_name in csv_name_arr:
        gps_processed_df = read_csv_ais(csv_name)
        other_idx = get_final_dataframe(gps_processed_df, csv_name, other_idx, total_desire_num)
        if other_idx > total_desire_num:
            break
    print("finishing generation real AIS .............")

In [ ]:
wrapping_function_obtain_data(finish_finding_ships)

### plot testing

In [ ]:
# file_name = "synthetic"
howmany=5
using_file_indices = np.arange(howmany)


fig = plt.figure()
ax = fig.add_subplot(111) # 2D

ax.set_xlim(-2200, 2200)
ax.set_ylim(-2200, 2200)

ax.plot(0, 0, '>', ms = 10, color='b') # ego vehicle

for using_file_idx in using_file_indices:

    # using_file_idx = 22 # 22
    unpickled_df = pd.read_pickle("./dk_csv_20180101/aisdk_20180101_{}.pkl".format(str(using_file_idx)))
    # print(unpickled_df)

    if unpickled_df["label"][0] == "L":
        ax.plot(unpickled_df['x'], 
                unpickled_df['y'], color='r')
        ax.plot(unpickled_df['x'].iloc[-1], unpickled_df['y'].iloc[-1], '*', ms = 10, color='r')
    elif unpickled_df["label"][0] == "R": 
        ax.plot(unpickled_df['x'], 
                unpickled_df['y'], color='g')
        ax.plot(unpickled_df['x'].iloc[-1], unpickled_df['y'].iloc[-1], '*', ms = 10, color='g')

# if using_file_idx > 30:
#     break